In [26]:
import pandas as pd

In [27]:
df = pd.read_excel('shifts - InterCon.xlsx')
df

,id,Employee_id,start_time,end_time,shift_id
0,1,a,2019-08-01 14:00:00 pm,2019-08-01 16:00:00 pm,28
1,2,a,2019-08-01 16:00:00 pm,2019-08-01 18:00:00 pm,11
2,3,a,2019-08-01 18:00:00 pm,2019-08-01 20:00:00 pm,44
3,4,a,2019-08-02 14:00:00 pm,2019-08-02 15:00:00 pm,12
4,5,a,2019-08-03 8:00:00 am,2019-08-03 10:00:00 am,23
5,6,b,2019-08-03 8:00:00 am,2019-08-03 11:00:00 am,24
6,7,b,2019-08-04 13:00:00 pm,2019-08-04 14:00:00 pm,25
7,8,b,2019-08-04 14:00:00 pm,2019-08-04 16:00:00 pm,6
8,9,b,2019-08-04 18:00:00 pm,2019-08-04 22:00:00 pm,47
9,10,c,2019-08-02 14:00:00 pm,2019-08-02 15:00:00 pm,30


Expected Output:

{'a': [[28, 11, 44], [12], [23]], 'b': [[24], [25, 6], [47]], 'c': [[30], [23, 38, 45, 20], [49]]}

Logic: add a new column to the df (for different employees - segment the employees first) with the window number (for the back to back shifts) and then group by the window number to concatenate shift_id into the same list if they are continuous shifts.

In [28]:
#define a function back_to_back for df so that we can use the function easily in the future:
def back_to_back(df):
    
#create an empty dictionary d to store the key and value pairs for each employee and their shifts:
    d = {}

#assign an intial window number 1 to a window variable w:
    w = 1

    for e in list(df['Employee_id'].unique()):
        #remember to reset_index because when you use .loc, the index should be the row index instead of the position
        employee = df[df.Employee_id == e].reset_index(drop = True)
        for i in range(len(employee)):
            #if the end time value is in the start time column (means it's a fake end time), then they should be the same window with same window number assigned:
            if employee.end_time.values[i] in employee.start_time.tolist():
                employee.loc[i, 'shift'] = w
            #if the end time value is not in the start time column (means the shift really ends), then they should be different window (a unique window):
            else:
                employee.loc[i, 'shift'] = w
                w += 1
        
        #groupby the window number to allocate different shift_id into different lists and then combine all the lists into 1 single list:
        v = employee.groupby('shift').shift_id.apply(list).reset_index().shift_id.tolist()
        
        #finnally add a value to the key in the empty dictionary:
        d[e] = v
    return d

In [29]:
back_to_back(df)

{'a': [[28, 11, 44], [12], [23]],
 'b': [[24], [25, 6], [47]],
 'c': [[30], [23, 38, 45, 20], [49]]}